# 1. Import necessary packages

In [1]:
import cv2
import numpy as np
import numpy as np
from PIL import Image
import os

# 2. Preparing our face-detection model

In [2]:
from facenet_pytorch.models.mtcnn import MTCNN
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device)


def mtcnn_detect(img: np.ndarray) -> np.ndarray:
    boxes, probs = mtcnn.detect(img)
#     boxes = bounding boxes around an image, probs = the confidence score of it being a face
    img = []
    
    if boxes is not None:
        for box in boxes :
            x_left = int(min(box[0], box[2]))
            x_right = int(max(box[0], box[2]))
            y_left = int(min(box[1], box[3]))
            y_right = int(max(box[1], box[3]))
            img.append([x_left, y_left, x_right, y_right])
            
    # returning the collection of detected faces along with their confidence scores
    return (img, probs)

In [3]:
# Face extractor method
def face_extractor(img):
    facesBoxes, confidenceScores = mtcnn_detect(frame)
    
    if facesBoxes is []:
        return []
    
    croppedFaces = []
    for (facesBox, score) in zip(facesBoxes, confidenceScores):
        if(score > 0.95):
            x1, y1, x2, y2 = facesBox
            croppedFaces.append([x1, y1, x2, y2])
            
    return croppedFaces

# 3. Loading the model

In [4]:
from keras.models import load_model
import pickle as pickle

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
vggModel = load_model("Models&Binarizer/vgg_fine.keras")
dnnModel = load_model("Models&Binarizer/dnn_fine.keras")

2023-12-01 13:10:32.893094: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2023-12-01 13:10:33.047677: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2023-12-01 13:10:33.093474: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2023-12-01 13:10:33.407822: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2023-12-01 13:10:33.454943: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.


In [6]:
# Load the LabelBinarizer from the file
with open('Models&Binarizer/label_binarizer.pkl', 'rb') as file:
    labelBinarizer = pickle.load(file)

/home/mukesh/MCA 3 Sem/MiniProject/MainFile/miniProject/lib/python3.11/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# 4. Working with webcamera/video

In [7]:
def predictImg(X):
    images_arr = X.astype('float32')
    images_arr = images_arr/np.max(images_arr)
    
    features = vggModel.predict(images_arr)
    features = features.reshape(-1, 25088)
    y = dnnModel.predict(features)
    maxProb = np.max(y)
    if(maxProb < 0.5) :
        return ["Unknown Face"]
    res = labelBinarizer.inverse_transform(y)
    return res

In [8]:
video_capture = cv2.VideoCapture(0) # check with 'testVideo2.mp4'

while True:
    _, frame = video_capture.read()
    
    facesCoordList = face_extractor(frame)
#     will return a list of ndarray(images)
    if facesCoordList is not []:
        for coordFace in facesCoordList:
            try:
                x1, y1, x2, y2 = coordFace
                face = frame[y1:y2, x1:x2]
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 125, 255), 2)
            
                face = cv2.resize(face, (224, 224))
# #             img = Image.fromarray(face, 'RGB')
                face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    
    #           Illumination Normalizing the image

                norm_img = np.zeros((face.shape[0], face.shape[1]))
                img_array = cv2.normalize(face, norm_img, 0, 255, cv2.NORM_MINMAX)

                img_array = np.array(face)
        
                img_arr = img_array.reshape(-1, 224, 224, 3)
        
                pred = predictImg(img_arr)
#               print(pred)
                cv2.putText(frame, str(pred[0]), (x1-5, y1-10), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
            except : 
                cv2.putText(frame, "exception", (x1-5, y1-10), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 25ms/step


QObject::moveToThread: Current thread (0xefe1dc0) is not the object's thread (0xebb6ed0).
Cannot move to target thread (0xefe1dc0)

QObject::moveToThread: Current thread (0xefe1dc0) is not the object's thread (0xebb6ed0).
Cannot move to target thread (0xefe1dc0)

QObject::moveToThread: Current thread (0xefe1dc0) is not the object's thread (0xebb6ed0).
Cannot move to target thread (0xefe1dc0)

QObject::moveToThread: Current thread (0xefe1dc0) is not the object's thread (0xebb6ed0).
Cannot move to target thread (0xefe1dc0)

QObject::moveToThread: Current thread (0xefe1dc0) is not the object's thread (0xebb6ed0).
Cannot move to target thread (0xefe1dc0)

QObject::moveToThread: Current thread (0xefe1dc0) is not the object's thread (0xebb6ed0).
Cannot move to target thread (0xefe1dc0)

QObject::moveToThread: Current thread (0xefe1dc0) is not the object's thread (0xebb6ed0).
Cannot move to target thread (0xefe1dc0)

QObject::moveToThread: Current thread (0xefe1dc0) is not the object's thread

1/1 [==============================] - 0s 23ms/step


In [10]:
video_capture.release()
cv2.destroyAllWindows()

In [11]:
path = "/home/mukesh/MCA 3 Sem/Trying out/MainFile/testingImages/second.jpg"
frame = cv2.imread(path)
facesCoordList = (face_extractor(frame))[0]
print(facesCoordList)

[492, 137, 723, 416]


In [8]:
x1, y1, x2, y2 = coordFace
face = frame[y1:y2, x1:x2]
cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 125, 255), 2)

face = cv2.resize(face, (224, 224))
# #             img = Image.fromarray(face, 'RGB')
face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
img_array = np.array(face)

#             Normalizing the image
#                 img_array = np.zeros((img.shape[0], img.shape[1]))
#                 img_array = cv2.normalize(img, norm_img, 0, 255, cv2.NORM_MINMAX)

img_arr = img_array.reshape(-1, 224, 224, 3)

pred = predictImg(img_arr)
#               print(pred)
cv2.putText(frame, str(pred[0]), (x1-5, y1-10), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)

NameError: name 'predictImg' is not defined

In [13]:
cv2.imshow("image", frame)
cv2.waitKey(0)

QObject::moveToThread: Current thread (0xd738980) is not the object's thread (0xd1e6ee0).
Cannot move to target thread (0xd738980)

QObject::moveToThread: Current thread (0xd738980) is not the object's thread (0xd1e6ee0).
Cannot move to target thread (0xd738980)

QObject::moveToThread: Current thread (0xd738980) is not the object's thread (0xd1e6ee0).
Cannot move to target thread (0xd738980)

QObject::moveToThread: Current thread (0xd738980) is not the object's thread (0xd1e6ee0).
Cannot move to target thread (0xd738980)

QObject::moveToThread: Current thread (0xd738980) is not the object's thread (0xd1e6ee0).
Cannot move to target thread (0xd738980)

QObject::moveToThread: Current thread (0xd738980) is not the object's thread (0xd1e6ee0).
Cannot move to target thread (0xd738980)

QObject::moveToThread: Current thread (0xd738980) is not the object's thread (0xd1e6ee0).
Cannot move to target thread (0xd738980)

QObject::moveToThread: Current thread (0xd738980) is not the object's thread

113